# Process cisbp files

In [1]:
library(data.table)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
motifsFile<-"../input/cisbp_Homo_sapiens_2023_06/RBP_Information_all_motifs.txt"
pwmsDir<-"../input/cisbp_Homo_sapiens_2023_06/pwms_all_motifs/"
cisbpFile<-"../input/cisbp_Homo_sapiens_2023_06/all_cisbp_mosbat.txt"

In [3]:
all_motifs<-fread(motifsFile,data.table = F) %>% as.data.frame()

In [4]:
motif_rbp<-all_motifs %>%
            select(Motif_ID,DBID,ENSDBID,RBP_Name) %>%
            dplyr::filter(Motif_ID!=".") %>%
            arrange(Motif_ID) %>%
            #mutate(RBP=paste(RBP_Name,ENSDBID,sep="__")) %>%
            select(-ENSDBID,-RBP_Name) %>%
            distinct()
head(motif_rbp)

,Motif_ID,DBID
,<chr>,<chr>
1,M001_0.6,RNCMPT00001
2,M002_0.6,RNCMPT00002
3,M004_0.6,RNCMPT00004
4,M012_0.6,RNCMPT00012
5,M013_0.6,RNCMPT00013
6,M016_0.6,RNCMPT00016


In [5]:
pwms.files <- paste0(pwmsDir,unique(motif_rbp$Motif_ID),".txt")
pwms.files <- pwms.files[file.size(pwms.files) != 0L]

In [6]:
pwms<- pwms.files %>%
        lapply(.,function(fname){
            mat<-read.table(fname,header = T) %>%
                 mutate(Motif_ID=sub("\\.txt","",basename(fname))) 
            return(mat)
        }) %>%
        do.call(rbind,.) %>%
        group_by(Motif_ID) %>%
        tidyr::nest(data = Pos:U) %>%
        arrange(Motif_ID)

In [7]:
dim(pwms)

[1] 175   2

In [8]:
pwms<-inner_join(pwms,motif_rbp) %>%
        distinct() 

Joining, by = "Motif_ID"


In [10]:
fout<-file(cisbpFile,open="at")
for(motif in unique(pwms$Motif_ID)){
    print(motif)
    pwms.motif<-pwms %>% filter(Motif_ID==motif)
    pfm<-pwms.motif$data[[1]]
    
    writeLines(c(paste0("Motif\t",pwms.motif$Motif_ID),
                 paste(colnames(pfm),collapse="\t"),
                 apply(pfm,1,function(row){paste(row,collapse="\t")}),
                 ""), 
               fout)
}
close(fout)

[1] "M001_0.6"
[1] "M002_0.6"
[1] "M004_0.6"
[1] "M012_0.6"
[1] "M013_0.6"
[1] "M016_0.6"
[1] "M017_0.6"
[1] "M019_0.6"
[1] "M020_0.6"
[1] "M021_0.6"
[1] "M022_0.6"
[1] "M023_0.6"
[1] "M024_0.6"
[1] "M025_0.6"
[1] "M026_0.6"
[1] "M027_0.6"
[1] "M031_0.6"
[1] "M032_0.6"
[1] "M033_0.6"
[1] "M035_0.6"
[1] "M036_0.6"
[1] "M037_0.6"
[1] "M040_0.6"
[1] "M042_0.6"
[1] "M043_0.6"
[1] "M044_0.6"
[1] "M046_0.6"
[1] "M047_0.6"
[1] "M048_0.6"
[1] "M049_0.6"
[1] "M050_0.6"
[1] "M051_0.6"
[1] "M052_0.6"
[1] "M053_0.6"
[1] "M054_0.6"
[1] "M055_0.6"
[1] "M056_0.6"
[1] "M061_0.6"
[1] "M062_0.6"
[1] "M065_0.6"
[1] "M068_0.6"
[1] "M069_0.6"
[1] "M070_0.6"
[1] "M071_0.6"
[1] "M072_0.6"
[1] "M073_0.6"
[1] "M074_0.6"
[1] "M075_0.6"
[1] "M077_0.6"
[1] "M079_0.6"
[1] "M081_0.6"
[1] "M082_0.6"
[1] "M083_0.6"
[1] "M085_0.6"
[1] "M086_0.6"
[1] "M087_0.6"
[1] "M088_0.6"
[1] "M089_0.6"
[1] "M102_0.6"
[1] "M103_0.6"
[1] "M104_0.6"
[1] "M105_0.6"
[1] "M106_0.6"
[1] "M108_0.6"
[1] "M109_0.6"
[1] "M111_0.6"
[1] "M112_